# Q-learning en el ambiente del Frozen Lake

<div style="text-align: right"><b>Autora</b>: Alicia Muñiz Jiménez<div>


La clave para resolver problemas de aprendizaje por refuerzo (RL) es encontrar la política óptima ($\pi^{*}$) de la función de valor para el problema específico. El Q-learning es un algoritmo del tipo modelo libre del RL basado en valor y es un policy off-learner, es decir, depende del ensayo y error para actualizar su experiencia y conocimiento del entorno, ya que tienen que aprender la dinámica del sistema a través de la experiencia, y aprende el valor de la política óptima independientemente del tipo de acciones del agente, así como que las actualizaciones de la función de valor se basan en la ecuación de Bellman  (Jang, Kim, Harerimana y Kim, 2019). 


Su Q proviene de quality (cualidad en inglés), ya que el Q-learning representa qué tan útil es una acción en ganar una recompensa a futuro (Shyalika, 2019). Es así que el Q-learning estima la función $Q(s,a)$ que es el valor esperado de hacer una acción 'a' en un estado 's' para estimar la política óptima y la ecuación de actualización de los valores $Q(s,a)$ es la siguiente: 


$$
Q(s, a) = Q(s, a) + \alpha [ r(s, a) + γmax Q' (s',a') - Q(s, a) ]
$$


Donde: 

*   $r(s,a)$ = recompensa obetenida en ese estado dada una acción.
*   $γ$ = factor de descuento para el estado futuro (al que se transitará), su valor es fijo y puede tomar el rango de valores de [0,1].
*   $α$ = parámetro de aprendizaje, no es fijo ya que al inicio comienza en 1 y con cada iteración irá disminuyendo su valor hasta alcanzar el valor de 0 o un límite inferior definido de antemano (ej. 0.01).
*   $Q(s, a)$ = valor Q del estado actual dada la acción a.
*   $max Q'(s', a')$ = máxima futura recompesa esperada del estado nuevo al que se transitará.


Como se puede apreciar en la ecuación anterior, el Q-learning usa el método de diferencias temporales para la actualización o estimación de los valores Q, ya que sólo toma en cuenta el estado actual y el siguiente estado para estimar los valores Q (Matiisen, s. f.). Por otro lado, cuando el Q-learning es ejecutado se crea lo que se llama una tabla-Q, que es una matriz donde las filas normalmente representan cada estado posible y las columnas las acciones posibles en cada estado (Aggarwal, s.f. ). Por lo tanto, cada entrada de la matriz es un par estado-acción. La tabla siempre se inicializa con todos los valores en 0 y se actualiza cada entrada con la ecuación de actualización de los valores Q, de acuerdo con lo que el agente explora en cada iteración. La tabla al final sirve como referencia para que el agente pueda seleccionar la mejor acción basada en los valores Q.

Finalmente, es importante recalcar que en el Q-learning el agente trabaja con una política epsilon-greedy, mas esta política no tiene valores fijos, sino que al inicio el algoritmo tiene un  epsilon de 1 y con cada iteración este valor va disminuyendo hasta terminar en 0 o en un límite inferior como 0.01, si es que así se especifica de antemano (Matiisen, s.f.). Esto se debe a que como el algoritmo no tiene conocimiento de la dinámica del sistema al inicio, entonces necesita explorar mucho para conocer el entorno, pero conforme va conociendo más del entorno puede explotar cada vez más y no solo explorar. 

Después de la breve introducción al Q-learning, ahora se procederá con un ejemplo práctico sobre dos agentes agentes que interactúan en el entorno virtual "Frozen Lake" del gimnasio OpenAI, esto para entender mejor el Q-learning y las diferencias que se pueden observar al compararlo con un agente que tiene una política de elección de comportamiento de acciones aleatorias. 



## Explicación del ambiente

La documentación de la paquetería *gymnasium* describe las sigueintes características del ambiente *Frozen Lake* (Farama Foundation, 2023): 

* **Estado de acciones (4):**

    - 0 = Izquierda
    - 1 = Abajo
    - 2 = Derecha
    - 3 = Arriba
    
* **Espacio observacional (16):** Se utilizará el ambiente de Frozen Lake con una cuadrícula de tamaño 4x4, por lo que el total de observaciones posibles (estados) son 16. 


* **Recompenzas:**

     - +1 si el agente alcanza la meta.
     - 0 si el agente cae en alún hoyo en el hielo.
     - 0 si el agente pasa por estados donde hay hielo congelado. 
     


# Código

**Instalación e importación de librerías**

In [34]:
import numpy as np
import gymnasium as gym 
import math, random

# Agente con acciones aleatorias

In [35]:

env=gym.make("FrozenLake-v1",desc=["SFFF", "FHFF", "FFFH", "FFFG"], is_slippery=False, render_mode="human")


In [36]:
ensayos = 10

for ensayos in range(1, ensayos+1):
  estado = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    _, reward, done, _, _ = env.step(action)

  print('Ensayo:{} action:{}'.format(ensayos, action))

env.close()


Ensayo:1 action:2
Ensayo:2 action:2
Ensayo:3 action:2
Ensayo:4 action:2
Ensayo:5 action:2
Ensayo:6 action:2
Ensayo:7 action:2
Ensayo:8 action:3
Ensayo:9 action:2
Ensayo:10 action:2


## Agente con el algorito de Q-learning

Se usó como base el código de Brooker (2020) sobre su agente de Q-learning en el ambiente de CartPole-v1, pero se adaptó para el ambiente Frozen Lake y el tipo de variables que maneja este ambiente. Además se implementaron cambios respecto al ambiente original, para fines didácticos y cambios en el código para adaptarlo a las nuevas actualizaciones de la paquetería gymnasium. De igual forma, se agregó una etapa de prueba al final para comprobar qué tan bien aprendió el agente. 

In [48]:

import numpy as np
env=gym.make("FrozenLake-v1",desc=["SFFF", "FHFF", "FFFH", "FFFG"], is_slippery=False, render_mode="ansi")


In [49]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

Q_table= np.zeros((state_space_size, action_space_size))
Q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [50]:
env.action_space.n

4

In [51]:
env.observation_space.n

16

In [52]:
Q_table.shape

def policy( state : tuple ):
    """Choosing action based on epsilon-greedy policy"""
    return np.argmax(Q_table[state])

def new_Q_value( reward : float ,  new_state : tuple , discount_factor=1 ) -> float:
    """Temporal diffrence for updating Q-value of state-action pair"""
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

# Adaptive learning of Learning Rate
def learning_rate(n : int , min_rate=0.01 ) -> float  :
    """Decaying learning rate"""
    return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))

def exploration_rate(n : int, min_rate= 0.1 ) -> float :
    """Decaying exploration rate"""
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

Q_table.shape

(16, 4)

In [53]:
env.reset()[1]

{'prob': 1}

In [54]:
num_episodio = []
puntajes_ep = []

n_episodes = 2000
for e in range(1, n_episodes+1):
    
    # Discretize state into buckets
    current_state = env.reset()[0]
    done = False
    num_episodio.append(e)
    score = 0

    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _, _ = env.step(action)
        #print(obs)
        new_state = obs
        
        score += reward

        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward, new_state )
        old_value = Q_table[current_state, action]
        Q_table[current_state, action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state

    print('Episode:{} Puntaje:{} e:{}'.format(e, score, exploration_rate(e)))
    puntajes_ep.append(score)



Episode:1 Puntaje:0.0 e:1
Episode:2 Puntaje:0.0 e:1
Episode:3 Puntaje:0.0 e:1
Episode:4 Puntaje:0.0 e:1
Episode:5 Puntaje:0.0 e:1
Episode:6 Puntaje:0.0 e:1
Episode:7 Puntaje:0.0 e:1
Episode:8 Puntaje:0.0 e:1
Episode:9 Puntaje:0.0 e:1
Episode:10 Puntaje:0.0 e:1
Episode:11 Puntaje:0.0 e:1
Episode:12 Puntaje:0.0 e:1
Episode:13 Puntaje:0.0 e:1
Episode:14 Puntaje:0.0 e:1
Episode:15 Puntaje:0.0 e:1
Episode:16 Puntaje:0.0 e:1
Episode:17 Puntaje:0.0 e:1
Episode:18 Puntaje:0.0 e:1
Episode:19 Puntaje:0.0 e:1
Episode:20 Puntaje:0.0 e:1
Episode:21 Puntaje:0.0 e:1
Episode:22 Puntaje:0.0 e:1
Episode:23 Puntaje:0.0 e:1
Episode:24 Puntaje:0.0 e:1
Episode:25 Puntaje:0.0 e:0.9829666607012196
Episode:26 Puntaje:0.0 e:0.9665762445130502
Episode:27 Puntaje:0.0 e:0.9507819773298184
Episode:28 Puntaje:0.0 e:0.9355420107730815
Episode:29 Puntaje:0.0 e:0.9208187539523752
Episode:30 Puntaje:0.0 e:0.906578314837765
Episode:31 Puntaje:0.0 e:0.8927900303521317
Episode:32 Puntaje:0.0 e:0.8794260687941501
Episode:33

In [55]:
Q_table

array([[4.85383214e-01, 9.99999980e-01, 1.43815434e-01, 3.59948534e-01],
       [4.89524813e-01, 0.00000000e+00, 2.37424057e-05, 2.81712324e-03],
       [6.04624099e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.88240744e-01, 9.99999997e-01, 0.00000000e+00, 5.06135743e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.13266049e-01, 1.00000000e+00, 4.84297879e-01, 6.58438116e-01],
       [9.36516736e-01, 9.99996281e-03, 0.00000000e+00, 0.00000000e+00],
       [4.68677038e-01, 1.98999049e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.76770597e-01, 4.87787526e-01, 1.00000000e+00, 6.18542014e-01],
       [5.00824296e-01, 5.61685783e-01, 1.00000000e

## Prueba de aprendizaje

In [59]:

import numpy as np
env=gym.make("FrozenLake-v1",desc=["SFFF", "FHFF", "FFFH", "FFFG"], is_slippery=False, render_mode="human")


In [60]:
n_episodes_t = 5
for e in range(1, n_episodes_t+1):
    
    # Siscretize state into buckets
    current_state = env.reset()[0]
    done = False
    score = 0

    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
         
        # act in the enviroment
        obs, reward, done, _,_= env.step(action)
        new_state = obs
        
        score += reward
      
        current_state = new_state


    print('Episodio:{} Puntaje:{}'.format(e, score))
    
env.close()

Episodio:1 Puntaje:1.0
Episodio:2 Puntaje:1.0
Episodio:3 Puntaje:1.0
Episodio:4 Puntaje:1.0
Episodio:5 Puntaje:1.0


**Referencias:** 

* Aggarwal, R. (s. f.). Q-learning - Reinforcement Learning. Recuperado el 15 de diciembre de 2022. https://miet.ac.in/assets/uploads/cs/Q%20Learning.pdf

* Brooker, R. (2020). OpenAI Gym: CartPole-v1 - Q-Learning [Youtube]. https://www.youtube.com/watch?v=JNKvJEzuNsc&t=227s

* Jang, B. Kim, M. Harerimana, G. y Kim, J.W. (2019). "Q-Learning Algorithms: A Comprehensive Classification and Applications,". IEEE Access, vol. 7,133653-133667, doi: 10.1109/ACCESS.2019.2941229

* Matiisen, T. (s. f.). Q-learning [Presentación]. Recuperado el 15 de diciembre de 2022. https://courses.cs.ut.ee/MTAT.03.292/2014_spring/uploads/Main/Q-learning.pdf

* Farama Foundation. (2023). Frozen Lake. Gymnasium Documentation. Recuperado el 28 de marzo de 2023. https://gymnasium.farama.org/environments/toy_text/frozen_lake/

* Shyalika, C. (2019). A Beginners Guide to Q-Learning. En Towards Data Science. Recuperado el 8 de diciembre de 2022. https://towardsdatascience.com/a-beginners-guide-to-q-learning-c3e2a30a653c
